## Load Library 

In [130]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

## Data Extraction

### 1 Indices MASI

In [3]:
dayofweek_mapping = {
    "Monday": "lundi",
    "Tuesday": "mardi",
    "Wednesday": "mercredi",
    "Thursday": "jeudi",
    "Friday": "vendredi",
    "Saturday": "samedi",
    "Sunday": "dimanche"
}
month_mapping = {
    'January': 'janvier',
    'February': 'février',
    'March': 'mars',
    'April': 'avril',
    'May': 'mai',
    'June': 'juin',
    'July': 'juillet',
    'August': 'août',
    'September': 'septembre',
    'October': 'octobre',
    'November': 'novembre',
    'December': 'décembre'
}
# Set up the Selenium driver
driver = webdriver.Chrome()

# Navigate to the webpage with the DatePicker component
driver.get("http://www.casablanca-bourse.com/fr/composition-des-indices")

# Define the start and end dates
start_date = pd.to_datetime("25/12/2022", format="%d/%m/%Y")
end_date = pd.to_datetime("25/06/2023", format="%d/%m/%Y")

# Create an empty dataframe to store the final combined data
df_MASI = pd.DataFrame()

# Loop through the dates
date = end_date
while date >= start_date:
    # Select the current date
    day_of_week = date.strftime("%A")
    month_name = date.strftime("%B")
    french_month = month_mapping.get(month_name)
    french_day_of_week = dayofweek_mapping.get(day_of_week)
    date_str = f"{french_day_of_week} {date.day} {french_month} {date.year}"
    if french_day_of_week not in ['samedi', 'dimanche'] and date_str not in ['lundi 1 mai 2023', 'lundi 24 avril 2023', 'mercredi 11 janvier 2023']:
        time.sleep(1)
        # Find the date input field
        date_input = driver.find_element(By.XPATH, "//input[@placeholder='Séance']")

        # Click on the date input field to open the calendar
        driver.execute_script("arguments[0].click();", date_input)
        time.sleep(1)
        
        if date_str in ['mardi 28 février 2023', 'mardi 31 janvier 2023']:
            left_element = driver.find_element(By.XPATH, "(//div[@class='flex flex-row items-center justify-between h-full px-2 py-2 rounded-lg']//button)[1]")
            driver.execute_script("arguments[0].click();", left_element)
        time.sleep(1)
            
        date_element = driver.find_element(By.XPATH, f"//div[@aria-label='Choose {date_str}']")
        driver.execute_script("arguments[0].click();", date_element)

        time.sleep(1)

        app = driver.find_element(By.XPATH, "//button[text()='Appliquer']")
        driver.execute_script("arguments[0].click();", app)
        time.sleep(3)
        # Perform actions to extract the data frame for the current date
        # Find the first table and extract its data
        first_table = driver.find_element(By.XPATH, "//table[@class='w-full print:max-w-full']")
        first_table_html = first_table.get_attribute('outerHTML')
        df_first_table = pd.read_html(first_table_html)[0]
        time.sleep(2)
        # Find and click the second button
        button = driver.find_element(By.XPATH, "//button[text()='2']")
        driver.execute_script("arguments[0].click();", button)

        # Wait for the tables to load
        time.sleep(3)

        # Find the second table and extract its data
        second_table = driver.find_element(By.XPATH, "//table[@class='w-full print:max-w-full']")
        second_table_html = second_table.get_attribute('outerHTML')
        df_second_table = pd.read_html(second_table_html)[0]

        # Concatenate the two DataFrames into one
        df_combined = pd.concat([df_first_table, df_second_table], ignore_index=True)

        # Add the same date value to the 'date' column for all rows
        df_combined['Date'] = date
        # Store or process the extracted data frame for the current date


        # Print or do something with the data frame
        print(f"Data frame for {date_str}:")
        print(df_combined)
        df_MASI = df_MASI.append(df_combined, ignore_index=True)

    # Move to the next date
    date -= pd.DateOffset(days=1)

# Close the browser
driver.quit()


Data frame for vendredi 23 juin 2023:
   Ticker               Instrument Nombre de titres Cours (MAD)  \
0     DLM  DELATTRE LEVIVIER MAROC        1 750 000        4000   
1     AGM                     AGMA          200 000    6 000,00   
2     CDM                      CDM       10 881 214       74700   
3     MLE            MAROC LEASING        2 776 768       35600   
4     NEJ               AUTO NEJMA        1 023 264    1 692,00   
..    ...                      ...              ...         ...   
95    MAB               MAGHREBAIL        1 384 182       94000   
96    MLE            MAROC LEASING        2 776 768       35600   
97    MOX          MAGHREB OXYGENE          812 500       20865   
98    NEJ               AUTO NEJMA        1 023 264    1 692,00   
99    OUL                   OULMES        1 980 000    1 590,00   

      Capitalisation       Date  
0      70 000 000,00 2023-06-23  
1   1 200 000 000,00 2023-06-23  
2   8 128 266 858,00 2023-06-23  
3     988 529 408,00 

Data frame for vendredi 16 juin 2023:
   Ticker               Instrument Nombre de titres Cours (MAD)  \
0     ADH        DOUJA PROM ADDOHA      402 551 254        1033   
1     ADI                ALLIANCES       22 078 588        7545   
2     ATL             ATLANTASANAD       60 283 595       12500   
3     ATW        ATTIJARIWAFA BANK      215 140 839       44000   
4     BCP                      BCP      203 312 473       24800   
..    ...                      ...              ...         ...   
69    ARD           ARADEI CAPITAL       11 213 964       44200   
70    TGC                 TGCC S.A       31 639 850       14730   
71    DYT       DISTY TECHNOLOGIES          998 110       19300   
72    AKT                  AKDITAL       12 666 670       32000   
73    DLM  DELATTRE LEVIVIER MAROC        1 750 000        4000   

       Capitalisation       Date  
0    4 158 354 453,82 2023-06-16  
1    1 665 829 464,60 2023-06-16  
2    7 535 449 375,00 2023-06-16  
3   94 661 969 16

Data frame for mercredi 7 juin 2023:
   Ticker         Instrument Nombre de titres Cours (MAD)     Capitalisation  \
0     ADH  DOUJA PROM ADDOHA      402 551 254         874   3 518 297 959,96   
1     ADI          ALLIANCES       22 078 588        6950   1 534 461 866,00   
2     ATL       ATLANTASANAD       60 283 595       12960   7 812 753 912,00   
3     ATW  ATTIJARIWAFA BANK      215 140 839       42530  91 499 398 826,70   
4     BCP                BCP      203 312 473       23795  48 378 202 950,35   
..    ...                ...              ...         ...                ...   
95    MAB         MAGHREBAIL        1 384 182       93700   1 296 978 534,00   
96    MLE      MAROC LEASING        2 776 768       35355     981 726 326,40   
97    MOX    MAGHREB OXYGENE          812 500       18600     151 125 000,00   
98    NEJ         AUTO NEJMA        1 023 264    1 692,00   1 731 362 688,00   
99    OUL             OULMES        1 980 000    1 300,00   2 574 000 000,00   

  

Data frame for lundi 29 mai 2023:
   Ticker               Instrument Nombre de titres Cours (MAD)  \
0     ADH        DOUJA PROM ADDOHA      402 551 254         735   
1     ADI                ALLIANCES       22 078 588        6700   
2     ATL             ATLANTASANAD       60 283 595       12950   
3     ATW        ATTIJARIWAFA BANK      215 140 839       42500   
4     BCP                      BCP      203 312 473       23600   
..    ...                      ...              ...         ...   
69    ARD           ARADEI CAPITAL       11 213 964       44960   
70    TGC                 TGCC S.A       31 639 850       13800   
71    DYT       DISTY TECHNOLOGIES          998 110       20000   
72    AKT                  AKDITAL       12 666 670       31010   
73    DLM  DELATTRE LEVIVIER MAROC        1 750 000        4000   

       Capitalisation       Date  
0    2 958 751 716,90 2023-05-29  
1    1 479 265 396,00 2023-05-29  
2    7 806 725 552,50 2023-05-29  
3   91 434 856 575,00

Data frame for mardi 23 mai 2023:
   Ticker         Instrument Nombre de titres Cours (MAD)     Capitalisation  \
0     ADH  DOUJA PROM ADDOHA      402 551 254         720   2 898 369 028,80   
1     ADI          ALLIANCES       22 078 588        6520   1 439 523 937,60   
2     ATL       ATLANTASANAD       60 283 595       12710   7 662 044 924,50   
3     ATW  ATTIJARIWAFA BANK      215 140 839       40800  87 777 462 312,00   
4     BCP                BCP      203 312 473       23100  46 965 181 263,00   
..    ...                ...              ...         ...                ...   
95    MAB         MAGHREBAIL        1 384 182       88000   1 218 080 160,00   
96    MLE      MAROC LEASING        2 776 768       34000     944 101 120,00   
97    MOX    MAGHREB OXYGENE          812 500       23980     194 837 500,00   
98    NEJ         AUTO NEJMA        1 023 264    1 692,00   1 731 362 688,00   
99    OUL             OULMES        1 980 000    1 322,00   2 617 560 000,00   

     

...

Data frame for jeudi 9 février 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18700   
2     ATL          ATLANTASANAD       60 283 595       11800   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1200   
4     SNP                  SNEP        2 400 000       60000   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       46320   
96    OUL                OULMES        1 980 000    1 433,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 230,00   

       Capitalisation       Date  
0      504 618 283,00 2023-02-09  
1       56 305 700,00 2023-02-09  
2    7 113 464 210,00 2023-02-09  
3      110 341 800,00 2023-02-09  
4    1 440 000 000,

Data frame for mardi 31 janvier 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       11790   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1281   
4     SNP                  SNEP        2 400 000       54000   
..    ...                   ...              ...         ...   
69    INV               INVOLYS          382 716        9505   
70    HPS                   HPS          703 599    6 200,00   
71    DWY                DISWAY        1 885 762       65600   
72    SOT               SOTHEMA        7 200 000    1 250,00   
73    PRO       PROMOPHARM S.A.        1 000 000    1 026,00   

      Capitalisation       Date  
0     504 618 283,00 2023-01-31  
1      54 499 100,00 2023-01-31  
2   7 107 435 850,50 2023-01-31  
3     117 789 871,50 2023-01-31  
4   1 296 000 000,00 20

Data frame for vendredi 20 janvier 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       75700   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       11400   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1278   
4     SNP                  SNEP        2 400 000       53600   
..    ...                   ...              ...         ...   
69    INV               INVOLYS          382 716       10000   
70    HPS                   HPS          703 599    6 200,00   
71    DWY                DISWAY        1 885 762       65810   
72    SOT               SOTHEMA        7 200 000    1 320,00   
73    PRO       PROMOPHARM S.A.        1 000 000    1 068,00   

      Capitalisation       Date  
0     490 367 189,00 2023-01-20  
1      54 499 100,00 2023-01-20  
2   6 872 329 830,00 2023-01-20  
3     117 514 017,00 2023-01-20  
4   1 286 400 000,00

Data frame for mardi 10 janvier 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       69830   
1     TIM                 TIMAR          301 100       18300   
2     ATL          ATLANTASANAD       60 283 595       12000   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1281   
4     SNP                  SNEP        2 400 000       52390   
..    ...                   ...              ...         ...   
69    INV               INVOLYS          382 716       10385   
70    HPS                   HPS          703 599    6 089,00   
71    DWY                DISWAY        1 885 762       62900   
72    SOT               SOTHEMA        7 200 000    1 289,00   
73    PRO       PROMOPHARM S.A.        1 000 000    1 068,00   

      Capitalisation       Date  
0     452 342 679,10 2023-01-10  
1      55 101 300,00 2023-01-10  
2   7 234 031 400,00 2023-01-10  
3     117 789 871,50 2023-01-10  
4   1 257 360 000,00 20

Data frame for vendredi 30 décembre 2022:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       78900   
1     TIM                 TIMAR          301 100       18000   
2     ATL          ATLANTASANAD       60 283 595       12500   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1571   
4     SNP                  SNEP        2 400 000       62000   
..    ...                   ...              ...         ...   
69    INV               INVOLYS          382 716       10395   
70    HPS                   HPS          703 599    6 500,00   
71    DWY                DISWAY        1 885 762       67000   
72    SOT               SOTHEMA        7 200 000    1 320,00   
73    PRO       PROMOPHARM S.A.        1 000 000    1 068,00   

      Capitalisation       Date  
0     511 096 053,00 2022-12-30  
1      54 198 000,00 2022-12-30  
2   7 535 449 375,00 2022-12-30  
3     144 455 806,50 2022-12-30  
4   1 488 000 000,0

In [4]:
df_final = df_MASI.copy()

In [5]:
sorted(list(df_final.Ticker.unique()))

['ADH',
 'ADI',
 'AFI',
 'AFM',
 'AGM',
 'AKT',
 'ALM',
 'ARD',
 'ATH',
 'ATL',
 'ATW',
 'BAL',
 'BCI',
 'BCP',
 'BOA',
 'CDM',
 'CIH',
 'CMA',
 'CMT',
 'COL',
 'CRS',
 'CSR',
 'CTM',
 'DHO',
 'DLM',
 'DRI',
 'DWY',
 'DYT',
 'EQD',
 'FBR',
 'GAZ',
 'HPS',
 'IAM',
 'IBC',
 'IMO',
 'INV',
 'JET',
 'LBV',
 'LES',
 'LHM',
 'LYD',
 'M2M',
 'MAB',
 'MDP',
 'MIC',
 'MLE',
 'MNG',
 'MOX',
 'MSA',
 'MUT',
 'NEJ',
 'NKL',
 'OUL',
 'PRO',
 'RDS',
 'REB',
 'RIS',
 'S2M',
 'SAH',
 'SBM',
 'SID',
 'SLF',
 'SMI',
 'SNA',
 'SNP',
 'SOT',
 'SRM',
 'STR',
 'TGC',
 'TIM',
 'TMA',
 'TQM',
 'UMR',
 'WAA',
 'ZDJ']

### 2 Informations sur l'émetteur

In [8]:
list_ticker = sorted(list(df_final.Ticker.unique()))
LYD_ticker = list_ticker.pop(40)
df_actionnaires_table_final = pd.DataFrame()
df_info_table_final = pd.DataFrame()
df_chiffres_table_final = pd.DataFrame()
df_ratios_table_final = pd.DataFrame()
# Set up the Selenium driver
driver = webdriver.Chrome()
for Index in list_ticker:


    # Open a website
    driver.get(f"https://www.casablanca-bourse.com/fr/live-market/instruments/{Index}")

    # Find and click the "Informations sur l'émetteur" link/button
    time.sleep(1)
    button = driver.find_element(By.LINK_TEXT, "Informations sur l'émetteur")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(3)
    page_source = driver.page_source


    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table or relevant elements in the parsed HTML
    tables = soup.find_all('table')
    
    if Index == 'DYT':
        # Extract firt
        df_actionnaires_table = pd.read_html(str(tables[0]), flavor='bs4')[0]

        # Set the columns names
        df_actionnaires_table.columns = ["Principaux actionnaires", "Percentage"]
        df_actionnaires_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_actionnaires_table)
        df_actionnaires_table_final = df_actionnaires_table_final.append(df_actionnaires_table, ignore_index=True)

        # Extract second table
        df_info_table = pd.read_html(str(tables[1]), flavor='bs4')[0]

        # Set the columns names
        df_info_table.columns = ["Informations sur la société", "value"]
        df_info_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_info_table)
        df_info_table_final = df_info_table_final.append(df_info_table, ignore_index=True)

        # Extract third table
        df_chiffres_table = pd.read_html(str(tables[2]), flavor='bs4')[0]

        # Set the columns names
        df_chiffres_table.columns = ["Chiffres clés", "2022", "2021"]
        df_chiffres_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_chiffres_table)
        df_chiffres_table_final = df_chiffres_table_final.append(df_chiffres_table, ignore_index=True)

        # Extract fourth table
        df_ratios_table = pd.read_html(str(tables[3]), flavor='bs4')[0]

        # Set the columns names
        df_ratios_table.columns = ["Ratios", "2022", "2021"]
        df_ratios_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_ratios_table)
        df_ratios_table_final = df_ratios_table_final.append(df_ratios_table, ignore_index=True)


    else:
        # Extract firt
        df_actionnaires_table = pd.read_html(str(tables[0]), flavor='bs4')[0]

        # Set the columns names
        df_actionnaires_table.columns = ["Principaux actionnaires", "Percentage"]
        df_actionnaires_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_actionnaires_table)
        df_actionnaires_table_final = df_actionnaires_table_final.append(df_actionnaires_table, ignore_index=True)

        # Extract second table
        df_info_table = pd.read_html(str(tables[1]), flavor='bs4')[0]

        # Set the columns names
        df_info_table.columns = ["Informations sur la société", "value"]
        df_info_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_info_table)
        df_info_table_final = df_info_table_final.append(df_info_table, ignore_index=True)

        # Extract third table
        df_chiffres_table = pd.read_html(str(tables[2]), flavor='bs4')[0]
        

        # Set the columns names
        df_chiffres_table.columns = ["Chiffres clés", "2022", "2021", "2020"]
        df_chiffres_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_chiffres_table)
        df_chiffres_table_final = df_chiffres_table_final.append(df_chiffres_table, ignore_index=True)

        # Extract fourth table
        df_ratios_table = pd.read_html(str(tables[3]), flavor='bs4')[0]

        # Set the columns names
        df_ratios_table.columns = ["Ratios", "2022", "2021", "2020"]
        df_ratios_table['Ticker'] = Index
        print(f"Data frame for {Index}:")
        print(df_ratios_table)
        df_ratios_table_final = df_ratios_table_final.append(df_ratios_table, ignore_index=True)


# Close the browser
driver.quit()

Data frame for ADH:
  Principaux actionnaires Percentage Ticker
0        Famille SEFRIOUI    64,11 %    ADH
1     DIVERS ACTIONNAIRES    30,81 %    ADH
2                    RCAR     5,07 %    ADH
3                   Total   100,00 %    ADH
Data frame for ADH:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                      DOUJA PROMOTION GROUPE ADDOHA    ADH  
1       Km 7. Route de Rabat . Ain Sebâa. Casablanca    ADH  
2                   A. Saaidi et Associés / Deloitte    ADH  
3                                         18/04/1988    ADH  
4                                         06/07/2006    ADH  
5                                                 12    ADH  
6  La société a pour objet : Directement.

Data frame for AGM:
            Principaux actionnaires Percentage Ticker
0                      ONA Courtage    50,03 %    AGM
1               DIVERS ACTIONNAIRES    31,54 %    AGM
2  PATRIMOINE GESTION ET PLACEMENTS    18,43 %    AGM
3                             Total   100,00 %    AGM
Data frame for AGM:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                               AGMA    AGM  
1  1. City Park Center. Rue Abderrahmane Sahraoui...    AGM  
2                 Price Watterhouse / Ernest & Young    AGM  
3                                         01/01/1964    AGM  
4                                         09/11/1998    AGM  
5                                                 12 

Data frame for ATH:
       Principaux actionnaires Percentage Ticker
0                        AMANA    58,30 %    ATH
1          DIVERS ACTIONNAIRES    15,72 %    ATH
2                         CIMR    15,04 %    ATH
3  HAKAM ABDELLATIF FINANCE SA    10,39 %    ATH
4              BENHAMMOU Karim     0,30 %    ATH
5            BENCHIMOL Salomon     0,24 %    ATH
6                    Personnel     0,00 %    ATH
7                        Total   100,00 %    ATH
Data frame for ATH:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                          AUTO HALL    ATH  
1                 64. Avenue Lalla Yacout Casablanca    ATH  
2       FIDAROC / FIDUCIAIRE DES SOCIETES MAROCAINES    ATH  
3     

Data frame for BCI:
          Principaux actionnaires Percentage Ticker
0  BNP PARIBAS IRB PARTICIPATIONS    66,74 %    BCI
1             DIVERS ACTIONNAIRES    16,57 %    BCI
2            AXA ASSURANCES MAROC     8,90 %    BCI
3                    ATLANTASANAD     7,80 %    BCI
4                           Total   100,00 %    BCI
Data frame for BCI:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                               BMCI    BCI  
1            26. Place des Nations Unies. Casablanca    BCI  
2           Price Waterhouse Coopers / Ernst & Young    BCI  
3                                         01/01/1964    BCI  
4                                         28/02/1972    BCI  
5          

Data frame for CDM:
     Principaux actionnaires Percentage Ticker
0  Holmarcom Finance Company    50,91 %    CDM
1       Crédit Agricole S.A.    15,00 %    CDM
2               ATLANTASANAD    12,80 %    CDM
3             WAFA ASSURANCE    10,74 %    CDM
4        DIVERS ACTIONNAIRES    10,55 %    CDM
5                      Total   100,00 %    CDM
Data frame for CDM:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                                CDM    CDM  
1              48-58 Boulevard Mohamed V. Casablanca    CDM  
2                                      Mazars et PwC    CDM  
3                                         08/02/1963    CDM  
4                                         03/05/1976    

...

Data frame for RIS:
  Principaux actionnaires Percentage Ticker
0             RMA WATANYA    36,74 %    RIS
1                Accor SA    33,34 %    RIS
2                    CIMR    12,32 %    RIS
3     DIVERS ACTIONNAIRES    10,18 %    RIS
4       Groupe MCMA-MAMDA     5,99 %    RIS
5               CFG Group     1,35 %    RIS
6         CDG performance     0,07 %    RIS
7                   Total   100,00 %    RIS
Data frame for RIS:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                              RISMA    RIS  
1  97, Boulevard Massira El Khadra, 5ème Etage, C...    RIS  
2                     Ernst & Young / Deloitte Touch    RIS  
3                                         01/01/199

Data frame for SID:
               Principaux actionnaires Percentage Ticker
0  Nouvelles Sidérurgies Industrielles    64,86 %    SID
1                  DIVERS ACTIONNAIRES    28,14 %    SID
2                                 RCAR     5,12 %    SID
3                       WAFA ASSURANCE     0,82 %    SID
4                                 MCMA     0,59 %    SID
5                    ATTIJARIWAFA BANK     0,27 %    SID
6                                MAMDA     0,21 %    SID
7                                Total   100,00 %    SID
Data frame for SID:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                            SONASID    SID  
1  Siège Social: N°2. El Aaroui- BP 551 à NADOR &...    SI

Data frame for SNP:
  Principaux actionnaires Percentage Ticker
0            Ynna Holding    62,92 %    SNP
1     DIVERS ACTIONNAIRES    37,08 %    SNP
2                   Total   100,00 %    SNP
Data frame for SNP:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0  Société Nationale d’Electrolyse et de Pétrochimie    SNP  
1              233. boulevard Mohammed V. Casablanca    SNP  
2            Deloitte Audit / Fidaroc Grant Thornton    SNP  
3                                         01/11/1973    SNP  
4                                         07/11/2007    SNP  
5                                                 12    SNP  
6  La creation et l'exploitation d'unites de prod...    SNP  
Data frame for SNP:
   

Data frame for TGC:
  Principaux actionnaires Percentage Ticker
0      BOUZOUBAA Mohammed    73,96 %    TGC
1     DIVERS ACTIONNAIRES    13,94 %    TGC
2      MC II Concrete Ltd    12,10 %    TGC
3                   Total   100,00 %    TGC
Data frame for TGC:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0     Travaux Généraux de Construction de Casablanca    TGC  
1                4 Rue Imam Mouslim Casablanca Oasis    TGC  
2            Audicis (Assist)/Fidaroc Grant Thornton    TGC  
3                                         02/10/1991    TGC  
4                                         16/12/2021    TGC  
5                                                 12    TGC  
6  la Société a pour objet tant au Maroc 

Data frame for UMR:
  Principaux actionnaires Percentage Ticker
0              SANAM AGRO    52,99 %    UMR
1           SANAM HOLDING    24,37 %    UMR
2     DIVERS ACTIONNAIRES    15,34 %    UMR
3        SANLAM ASSURANCE     6,08 %    UMR
4           ALJIA HOLDING     1,22 %    UMR
5                   Total   100,00 %    UMR
Data frame for UMR:
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value Ticker  
0                                             UNIMER    UMR  
1  Km 11. Rue L. Route des Zanatas Aïn Sebaâ Sidi...    UMR  
2  Cabinet Youssef Sebti / Cabinet Horwath Maroc ...    UMR  
3                                         01/01/1973    UMR  
4                                         29/03/2001    UMR  
5              

## Data Preprocessing

### 1 Indices MASI

In [30]:
df_final = df_MASI.copy()

In [10]:
df_final.dtypes

Ticker                      object
Instrument                  object
Nombre de titres            object
Cours (MAD)                 object
Capitalisation              object
Date                datetime64[ns]
dtype: object

In [11]:
df_final

,Ticker,Instrument,Nombre de titres,Cours (MAD),Capitalisation,Date
0,DLM,DELATTRE LEVIVIER MAROC,1 750 000,4000,"70 000 000,00",2023-06-23
1,AGM,AGMA,200 000,"6 000,00","1 200 000 000,00",2023-06-23
2,CDM,CDM,10 881 214,74700,"8 128 266 858,00",2023-06-23
3,MLE,MAROC LEASING,2 776 768,35600,"988 529 408,00",2023-06-23
4,NEJ,AUTO NEJMA,1 023 264,"1 692,00","1 731 362 688,00",2023-06-23
...,...,...,...,...,...,...
9683,INV,INVOLYS,382 716,10595,"40 548 760,20",2022-12-26
9684,HPS,HPS,703 599,"6 000,00","4 221 594 000,00",2022-12-26
9685,DWY,DISWAY,1 885 762,69490,"1 310 416 013,80",2022-12-26
9686,SOT,SOTHEMA,7 200 000,"1 350,00","9 720 000 000,00",2022-12-26


In [31]:
 # Transform into categorical type
df_final['Ticker'] = df_final['Ticker'].astype('category')
df_final['Instrument'] = df_final['Instrument'].astype('category')

In [32]:
 # Transform into float and int type
df_final['Nombre de titres'] = df_final['Nombre de titres'].str.replace(' ', '').astype(int)
df_final['Cours (MAD)'] = df_final['Cours (MAD)'].str.replace(' ', '').str.replace(',', '').str[:-2] + '.' + df_final['Cours (MAD)'].str[-2:]
df_final['Cours (MAD)'] = df_final['Cours (MAD)'].astype(float)
df_final['Capitalisation'] = df_final['Capitalisation'].str.replace(' ', '').str.replace(',', '.')
df_final['Capitalisation'] = df_final['Capitalisation'].astype(float)

In [33]:
df_final

,Ticker,Instrument,Nombre de titres,Cours (MAD),Capitalisation,Date
0,DLM,DELATTRE LEVIVIER MAROC,1750000,40.00,7.000000e+07,2023-06-23
1,AGM,AGMA,200000,6000.00,1.200000e+09,2023-06-23
2,CDM,CDM,10881214,747.00,8.128267e+09,2023-06-23
3,MLE,MAROC LEASING,2776768,356.00,9.885294e+08,2023-06-23
4,NEJ,AUTO NEJMA,1023264,1692.00,1.731363e+09,2023-06-23
...,...,...,...,...,...,...
9683,INV,INVOLYS,382716,105.95,4.054876e+07,2022-12-26
9684,HPS,HPS,703599,6000.00,4.221594e+09,2022-12-26
9685,DWY,DISWAY,1885762,694.90,1.310416e+09,2022-12-26
9686,SOT,SOTHEMA,7200000,1350.00,9.720000e+09,2022-12-26


In [149]:
df_final.to_csv('Bourses Casablanca/Indice_MASI.csv', index=False)

### 2 Informations sur l'émetteur

#### Actionnaires

In [39]:
df_a = df_actionnaires_table_final.copy()

In [156]:
df_a.dtypes

Principaux actionnaires    category
Percentage                  float64
Ticker                     category
dtype: object

In [155]:
df_a.iloc[10,0]

'SCR (Société Centrale de Réassurance)'

In [36]:
df_a

,Principaux actionnaires,Percentage,Ticker
0,Famille SEFRIOUI,"64,11 %",ADH
1,DIVERS ACTIONNAIRES,"30,81 %",ADH
2,RCAR,"5,07 %",ADH
3,Total,"100,00 %",ADH
4,NAFAKH LAZRAQ ALAMI MOHAMED,"51,68 %",ADI
...,...,...,...
543,AXA ASSURANCES MAROC,"17,58 %",ZDJ
544,AL HOCEINIA SA,"5,01 %",ZDJ
545,REBAB COMPANY,"4,57 %",ZDJ
546,DIVERS ACTIONNAIRES,"3,33 %",ZDJ


In [41]:
df_a = df_a[df_a['Principaux actionnaires'] != 'Total']
df_a['Percentage'] = df_a['Percentage'].str.replace(',', '.').str.replace(' ', '').str.replace('%', '')
df_a['Percentage'] = df_a['Percentage'].astype(float)

In [42]:
 # Transform into categorical type
df_a['Ticker'] = df_a['Ticker'].astype('category')
df_a['Principaux actionnaires'] = df_a['Principaux actionnaires'].astype('category')

In [44]:
df_a

,Principaux actionnaires,Percentage,Ticker
0,Famille SEFRIOUI,64.11,ADH
1,DIVERS ACTIONNAIRES,30.81,ADH
2,RCAR,5.07,ADH
4,NAFAKH LAZRAQ ALAMI MOHAMED,51.68,ADI
5,DIVERS ACTIONNAIRES,23.30,ADI
...,...,...,...
542,SOMED,69.50,ZDJ
543,AXA ASSURANCES MAROC,17.58,ZDJ
544,AL HOCEINIA SA,5.01,ZDJ
545,REBAB COMPANY,4.57,ZDJ


In [157]:
df_a.to_csv('Bourses Casablanca/Actionnaires.csv', index=False, encoding='utf-8')

#### Informations

In [43]:
df_i = df_info_table_final.copy()

In [45]:
df_i = df_i[df_i['Informations sur la société'] != 'Objet social']

In [ ]:
df_i['Ticker'] = df_i['Ticker'].astype('category')
df_i['Informations sur la société'] = df_i['Informations sur la société'].astype('category')

In [46]:
df_i

,Informations sur la société,value,Ticker
0,Nom de la société,DOUJA PROMOTION GROUPE ADDOHA,ADH
1,Siège social,Km 7. Route de Rabat . Ain Sebâa. Casablanca,ADH
2,Commissaire aux comptes,A. Saaidi et Associés / Deloitte,ADH
3,Date de constitution,18/04/1988,ADH
4,Date d'introduction,06/07/2006,ADH
...,...,...,...
512,Siège social,"Boulevard Sidi Mohamed Ben Abdellah, Tour Mari...",ZDJ
513,Commissaire aux comptes,Deloitte et Cabinet M. Youssef Sebti,ZDJ
514,Date de constitution,01/01/1929,ZDJ
515,Date d'introduction,20/06/1955,ZDJ


In [151]:
df_i.to_csv('Bourses Casablanca/Informations.csv', index=False)

#### Chiffres

In [119]:
df_c = df_chiffres_table_final.copy()

In [120]:
df_c

,Chiffres clés,2022,2021,2020,Ticker
0,Comptes consolidés,Oui,Oui,Oui,ADH
1,Capital social (2),"4 025 512 540,00","4 025 512 540,00","4 025 512 540,00",ADH
2,Capitaux propres (3),"9 643 388 563,00","9 605 304 598,00","9 858 347 232,00",ADH
3,Nombre titres(2),402 551 254,402 551 254,402 551 254,ADH
4,Chiffre d'Affaires,"1 377 780 080,00","1 196 552 867,00","1 173 466 044,00",ADH
...,...,...,...,...,...
513,Capitaux propres (3),"24 000 192,62","47 376 000,00","89 444 000,00",ZDJ
514,Nombre titres(2),572 849,572 849,572 849,ZDJ
515,Chiffre d'Affaires,"30 000,00","536 100 000,00","429 173 000,00",ZDJ
516,Résultat d'exploitation,"-365 243,94","-21 941 000,00","-23 808 000,00",ZDJ


In [121]:
df_c = df_c[df_c['Chiffres clés'] != 'Comptes consolidés']

In [122]:
df_c['2020'] = df_c['2020'].str.replace(',', '.').str.replace(' ', '')
df_c['2020'] = pd.to_numeric(df_c['2020'], errors='coerce')
df_c['2021'] = df_c['2021'].str.replace(',', '.').str.replace(' ', '')
df_c['2021'] = pd.to_numeric(df_c['2021'], errors='coerce')
df_c['2022'] = df_c['2022'].str.replace(',', '.').str.replace(' ', '')
df_c['2022'] = pd.to_numeric(df_c['2022'], errors='coerce')
df_c['2020'] = df_c['2020'].astype(float)
df_c['2021'] = df_c['2021'].astype(float)
df_c['2022'] = df_c['2022'].astype(float)

df_c['Chiffres clés'] = df_c['Chiffres clés'].str.replace('[\(2\), \(3\), \(4\)]', '')

C:\Users\user\AppData\Local\Temp/ipykernel_16224/2977745200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['2020'] = df_c['2020'].str.replace(',', '.').str.replace(' ', '')
C:\Users\user\AppData\Local\Temp/ipykernel_16224/2977745200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c['2020'] = pd.to_numeric(df_c['2020'], errors='coerce')
C:\Users\user\AppData\Local\Temp/ipykernel_16224/2977745200.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [ ]:
df_c['Ticker'] = df_c['Ticker'].astype('category')
df_c['Chiffres clés'] = df_c['Chiffres clés'].astype('category')

In [124]:
df_c.dtypes

Chiffres clés     object
2022             float64
2021             float64
2020             float64
Ticker            object
dtype: object

In [125]:
df_c

,Chiffres clés,2022,2021,2020,Ticker
1,Capitalsocial,4.025513e+09,4.025513e+09,4.025513e+09,ADH
2,Capitauxpropres,9.643389e+09,9.605305e+09,9.858347e+09,ADH
3,Nombretitres,4.025513e+08,4.025513e+08,4.025513e+08,ADH
4,Chiffred'Affaires,1.377780e+09,1.196553e+09,1.173466e+09,ADH
5,Résultatd'exploitation,1.091057e+08,-1.276460e+08,-8.131215e+08,ADH
...,...,...,...,...,...
513,Capitauxpropres,2.400019e+07,4.737600e+07,8.944400e+07,ZDJ
514,Nombretitres,5.728490e+05,5.728490e+05,5.728490e+05,ZDJ
515,Chiffred'Affaires,3.000000e+04,5.361000e+08,4.291730e+08,ZDJ
516,Résultatd'exploitation,-3.652439e+05,-2.194100e+07,-2.380800e+07,ZDJ


In [152]:
df_c.to_csv('Bourses Casablanca/Chiffres clés.csv', index=False)

#### Ratios

In [139]:
df_r = df_ratios_table_final.copy()

In [140]:
df_r

,Ratios,2022,2021,2020,Ticker
0,BPA,009,-022,-185,ADH
1,ROE (en %),039,-092,-755,ADH
2,Payout (en %),-,-,-,ADH
3,Dividend yield (en %),-,-,-,ADH
4,PER,6737,-,-,ADH
...,...,...,...,...,...
439,ROE (en %),-092,-500,-1954,ZDJ
440,Payout (en %),-,-,-,ZDJ
441,Dividend yield (en %),-,-,-,ZDJ
442,PER,-,-,-,ZDJ


In [141]:
df_r['2020'] = pd.to_numeric(df_r['2020'].replace('-', np.nan).str[:-2] + '.' + df_r['2020'].str[-2:], errors='coerce')
df_r['2021'] = pd.to_numeric(df_r['2021'].replace('-', np.nan).str[:-2] + '.' + df_r['2021'].str[-2:], errors='coerce')
df_r['2022'] = pd.to_numeric(df_r['2022'].replace('-', np.nan).str[:-2] + '.' + df_r['2022'].str[-2:], errors='coerce')


In [ ]:
df_r['Ticker'] = df_r['Ticker'].astype('category')
df_r['Ratios'] = df_r['Ratios'].astype('category')

In [142]:
df_r

,Ratios,2022,2021,2020,Ticker
0,BPA,0.09,-0.22,-1.85,ADH
1,ROE (en %),0.39,-0.92,-7.55,ADH
2,Payout (en %),NaN,NaN,NaN,ADH
3,Dividend yield (en %),NaN,NaN,NaN,ADH
4,PER,67.37,NaN,NaN,ADH
...,...,...,...,...,...
439,ROE (en %),-0.92,-5.00,-19.54,ZDJ
440,Payout (en %),NaN,NaN,NaN,ZDJ
441,Dividend yield (en %),NaN,NaN,NaN,ZDJ
442,PER,NaN,NaN,NaN,ZDJ


In [153]:
df_r.to_csv('Bourses Casablanca/Ratios.csv', index=False)

## Connect to Microsoft SQL Server 

In [162]:
import pyodbc
import sqlalchemy

In [168]:
server_name = 'DESKTOP-29ONFGJ'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};Trusted_Connection=yes;'

In [169]:
conn = pyodbc.connect(connection_string)

In [177]:
def Insert_MSS(df, table_name):
    
    # Establish a connection to your SQL Server database
    engine = sqlalchemy.create_engine('mssql+pyodbc://DESKTOP-29ONFGJ/master?driver=SQL+Server')

    # Create the table in the database (if it does not exist)
    df.head(0).to_sql(name=table_name, con=engine, if_exists='replace', index=False)  # Create an empty table

    # Insert the DataFrame into the database table
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

    # Close the database connection
    engine.dispose()


In [178]:
Insert_MSS(df_final, 'IndiceMASI')

In [179]:
Insert_MSS(df_a, 'Actionnaires')

In [180]:
Insert_MSS(df_i, 'Informations')

In [183]:
Insert_MSS(df_c, 'Chiffres')

In [182]:
Insert_MSS(df_r, 'Ratios')

In [170]:
##Create the table
CREATE TABLE IndiceMASI (
    Ticker NVARCHAR(10),
    Instrument NVARCHAR(50),
    [Nombre de titres] INT,
    [Cours (MAD)] FLOAT,
    Capitalisation BIGINT,
    [Date] DATE
);

SyntaxError: invalid syntax (Temp/ipykernel_16224/3104282137.py, line 2)

## Drafts

In [6]:
df_combined

,Ticker,Instrument,Nombre de titres,Cours (MAD),Capitalisation
0,DLM,DELATTRE LEVIVIER MAROC,1 750 000,4000,"70 000 000,00"
1,CTM,CTM,1 225 978,62400,"765 010 272,00"
2,MLE,MAROC LEASING,2 776 768,35600,"988 529 408,00"
3,NEJ,AUTO NEJMA,1 023 264,"1 692,00","1 731 362 688,00"
4,OUL,OULMES,1 980 000,"1 590,00","3 148 200 000,00"
...,...,...,...,...,...
69,CIH,CIH,30 519 784,36300,"11 078 681 592,00"
70,ATH,AUTO HALL,50 294 528,7500,"3 772 089 600,00"
71,ATW,ATTIJARIWAFA BANK,215 140 839,44395,"95 511 775 474,05"
72,DWY,DISWAY,1 885 762,72000,"1 357 748 640,00"


In [ ]:
dayofweek_mapping = {
    "Monday": "lundi",
    "Tuesday": "mardi",
    "Wednesday": "mercredi",
    "Thursday": "jeudi",
    "Friday": "vendredi",
    "Saturday": "samedi",
    "Sunday": "dimanche"
}
month_mapping = {
    'January': 'janvier',
    'February': 'février',
    'March': 'mars',
    'April': 'avril',
    'May': 'mai',
    'June': 'juin',
    'July': 'juillet',
    'August': 'août',
    'September': 'septembre',
    'October': 'octobre',
    'November': 'novembre',
    'December': 'décembre'
}
# Set up the Selenium driver
driver = webdriver.Chrome()

# Navigate to the webpage with the DatePicker component
driver.get("http://www.casablanca-bourse.com/fr/composition-des-indices")

# Define the start and end dates
start_date = pd.to_datetime("25/12/2022", format="%d/%m/%Y")
end_date = pd.to_datetime("25/05/2023", format="%d/%m/%Y")

# Create an empty dataframe to store the final combined data
df_final = pd.DataFrame()

# Loop through the dates
date = end_date
while date >= start_date:
    # Select the current date
    day_of_week = date.strftime("%A")
    month_name = date.strftime("%B")
    french_month = month_mapping.get(month_name)
    french_day_of_week = dayofweek_mapping.get(day_of_week)
    date_str = f"{french_day_of_week} {date.day} {french_month} {date.year}"
    if french_day_of_week not in ['samedi', 'dimanche'] and date_str not in ['lundi 1 mai 2023', 'lundi 24 avril 2023', 'mercredi 11 janvier 2023']:
        time.sleep(1)
        # Find the date input field
        date_input = driver.find_element(By.XPATH, "//input[@placeholder='Séance']")

        # Click on the date input field to open the calendar
        driver.execute_script("arguments[0].click();", date_input)
        time.sleep(1)
        
        if date_str in ['mardi 28 février 2023', 'mardi 31 janvier 2023']:
            left_element = driver.find_element(By.XPATH, "(//div[@class='flex flex-row items-center justify-between h-full px-2 py-2 rounded-lg']//button)[1]")
            driver.execute_script("arguments[0].click();", left_element)
        time.sleep(1)
            
        date_element = driver.find_element(By.XPATH, f"//div[@aria-label='Choose {date_str}']")
        driver.execute_script("arguments[0].click();", date_element)

        time.sleep(1)

        app = driver.find_element(By.XPATH, "//button[text()='Appliquer']")
        driver.execute_script("arguments[0].click();", app)
        time.sleep(3)
        # Perform actions to extract the data frame for the current date
        # Find the first table and extract its data
        first_table = driver.find_element(By.XPATH, "//table[@class='w-full print:max-w-full']")
        first_table_html = first_table.get_attribute('outerHTML')
        df_first_table = pd.read_html(first_table_html)[0]
        time.sleep(2)
        # Find and click the second button
        button = driver.find_element(By.XPATH, "//button[text()='2']")
        driver.execute_script("arguments[0].click();", button)

        # Wait for the tables to load
        time.sleep(3)

        # Find the second table and extract its data
        second_table = driver.find_element(By.XPATH, "//table[@class='w-full print:max-w-full']")
        second_table_html = second_table.get_attribute('outerHTML')
        df_second_table = pd.read_html(second_table_html)[0]

        # Concatenate the two DataFrames into one
        df_combined = pd.concat([df_first_table, df_second_table], ignore_index=True)

        # Add the same date value to the 'date' column for all rows
        df_combined['Date'] = date
        # Store or process the extracted data frame for the current date


        # Print or do something with the data frame
        print(f"Data frame for {date_str}:")
        print(df_combined)
        df_final = df_final.append(df_combined, ignore_index=True)

    # Move to the next date
    date -= pd.DateOffset(days=1)

# Close the browser
driver.quit()


In [44]:
dayofweek_mapping = {
    "Monday": "lundi",
    "Tuesday": "mardi",
    "Wednesday": "mercredi",
    "Thursday": "jeudi",
    "Friday": "vendredi",
    "Saturday": "samedi",
    "Sunday": "dimanche"
}
month_mapping = {
    'January': 'janvier',
    'February': 'février',
    'March': 'mars',
    'April': 'avril',
    'May': 'mai',
    'June': 'juin',
    'July': 'juillet',
    'August': 'août',
    'September': 'septembre',
    'October': 'octobre',
    'November': 'novembre',
    'December': 'décembre'
}
# Set up the Selenium driver
driver = webdriver.Chrome()

# Navigate to the webpage with the DatePicker component
driver.get("http://www.casablanca-bourse.com/fr/composition-des-indices")

# Define the start and end dates
start_date = pd.to_datetime("25/12/2022", format="%d/%m/%Y")
end_date = pd.to_datetime("25/06/2023", format="%d/%m/%Y")

# Loop through the dates
date = end_date
while date >= start_date:
    # Select the current date
    day_of_week = date.strftime("%A")
    month_name = date.strftime("%B")
    french_month = month_mapping.get(month_name)
    french_day_of_week = dayofweek_mapping.get(day_of_week)
    date_str = f"{french_day_of_week} {date.day} {french_month} {date.year}"
    if french_day_of_week not in ['samedi', 'dimanche'] and date_str not in ['lundi 1 mai 2023', 'lundi 24 avril 2023']:
        time.sleep(1)
        # Find the date input field
        date_input = driver.find_element(By.XPATH, "//input[@placeholder='Séance']")

        # Click on the date input field to open the calendar
        driver.execute_script("arguments[0].click();", date_input)
        time.sleep(1)
        if date_str in ['mardi 28 février 2023', 'mardi 31 janvier 2023']:
            left_element = driver.find_element(By.XPATH, "(//div[@class='flex flex-row items-center justify-between h-full px-2 py-2 rounded-lg']//button)[1]")
            driver.execute_script("arguments[0].click();", left_element)
        time.sleep(1)
        date_element = driver.find_element(By.XPATH, f"//div[@aria-label='Choose {date_str}']")
        driver.execute_script("arguments[0].click();", date_element)
        time.sleep(1)
        
        

        app = driver.find_element(By.XPATH, "//button[text()='Appliquer']")
        driver.execute_script("arguments[0].click();", app)
        time.sleep(2)
        # Perform actions to extract the data frame for the current date
        # ...

        # Add the same date value to the 'date' column for all rows
        df_combined['Date'] = date
        # Store or process the extracted data frame for the current date


        # Print or do something with the data frame
        print(f"Data frame for {date_str}:")
        print(df_combined)
        
    # Move to the next date
    date -= pd.DateOffset(days=1)

# Close the browser
driver.quit()


Data frame for vendredi 23 juin 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-06-23  
1       54 499 100,00 2023-06-23  
2    8 011 689 775,50 2023-06-23  
3      106 020 079,50 2023-06-23  
4    1 351 200 000

Data frame for mercredi 14 juin 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-06-14  
1       54 499 100,00 2023-06-14  
2    8 011 689 775,50 2023-06-14  
3      106 020 079,50 2023-06-14  
4    1 351 200 000

Data frame for lundi 5 juin 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-06-05  
1       54 499 100,00 2023-06-05  
2    8 011 689 775,50 2023-06-05  
3      106 020 079,50 2023-06-05  
4    1 351 200 000,00 

Data frame for jeudi 25 mai 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-05-25  
1       54 499 100,00 2023-05-25  
2    8 011 689 775,50 2023-05-25  
3      106 020 079,50 2023-05-25  
4    1 351 200 000,00 

Data frame for mardi 16 mai 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-05-16  
1       54 499 100,00 2023-05-16  
2    8 011 689 775,50 2023-05-16  
3      106 020 079,50 2023-05-16  
4    1 351 200 000,00 

...

Data frame for lundi 6 mars 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-03-06  
1       54 499 100,00 2023-03-06  
2    8 011 689 775,50 2023-03-06  
3      106 020 079,50 2023-03-06  
4    1 351 200 000,00 

Data frame for jeudi 23 février 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-02-23  
1       54 499 100,00 2023-02-23  
2    8 011 689 775,50 2023-02-23  
3      106 020 079,50 2023-02-23  
4    1 351 200 000

Data frame for mardi 14 février 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-02-14  
1       54 499 100,00 2023-02-14  
2    8 011 689 775,50 2023-02-14  
3      106 020 079,50 2023-02-14  
4    1 351 200 000

Data frame for vendredi 3 février 2023:
   Ticker            Instrument Nombre de titres Cours (MAD)  \
0     M2M             M2M Group          647 777       77900   
1     TIM                 TIMAR          301 100       18100   
2     ATL          ATLANTASANAD       60 283 595       13290   
3     SNA  STOKVIS NORD AFRIQUE        9 195 150        1153   
4     SNP                  SNEP        2 400 000       56300   
..    ...                   ...              ...         ...   
95    BCI                  BMCI       13 279 286       50200   
96    OUL                OULMES        1 980 000    1 376,00   
97    REB         REBAB COMPANY          176 456       11350   
98    ZDJ          ZELLIDJA S.A          572 849        8200   
99    WAA        WAFA ASSURANCE        3 500 000    3 700,00   

       Capitalisation       Date  
0      504 618 283,00 2023-02-03  
1       54 499 100,00 2023-02-03  
2    8 011 689 775,50 2023-02-03  
3      106 020 079,50 2023-02-03  
4    1 351 200 0

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@aria-label='Choose mardi 31 janvier 2023']"}
  (Session info: chrome=114.0.5735.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008AA813+48355]
	(No symbol) [0x0083C4B1]
	(No symbol) [0x00745358]
	(No symbol) [0x007709A5]
	(No symbol) [0x00770B3B]
	(No symbol) [0x0079E232]
	(No symbol) [0x0078A784]
	(No symbol) [0x0079C922]
	(No symbol) [0x0078A536]
	(No symbol) [0x007682DC]
	(No symbol) [0x007693DD]
	GetHandleVerifier [0x00B0AABD+2539405]
	GetHandleVerifier [0x00B4A78F+2800735]
	GetHandleVerifier [0x00B4456C+2775612]
	GetHandleVerifier [0x009351E0+616112]
	(No symbol) [0x00845F8C]
	(No symbol) [0x00842328]
	(No symbol) [0x0084240B]
	(No symbol) [0x00834FF7]
	BaseThreadInitThunk [0x751A00C9+25]
	RtlGetAppContainerNamedObjectPath 

In [3]:
# Set up the Selenium driver (adjust path and options as needed)
driver = webdriver.Chrome()

# Navigate to the webpage with the dynamic tables
driver.get("http://www.casablanca-bourse.com/fr/composition-des-indices")

page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find the table or relevant elements in the parsed HTML
tables = soup.find_all('table')

# Extract firt
df_first_table = pd.read_html(str(tables[0]), flavor='bs4')[0]

# Set the columns names
df_first_table.columns = ["Ticker", "Instrument", "Nombre de titres", "Cours (MAD)", "Capitalisation"]
print(df_first_table)
time.sleep(1)
# Find and click the second button
button = driver.find_element(By.XPATH, "//button[text()='2']")
driver.execute_script("arguments[0].click();", button)

# Wait for the tables to load
time.sleep(2)

page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find the table or relevant elements in the parsed HTML
tables = soup.find_all('table')

# Extract second
df_second_table = pd.read_html(str(tables[0]), flavor='bs4')[0]

# Set the columns names
df_second_table.columns = ["Ticker", "Instrument", "Nombre de titres", "Cours (MAD)", "Capitalisation"]
print(df_second_table)

# Concatenate the two DataFrames into one
df_combined = pd.concat([df_first_table, df_second_table], ignore_index=True)

# Print the combined DataFrame
print(df_combined)

# Close the browser
driver.quit()


   Ticker                     Instrument Nombre de titres Cours (MAD)  \
0     DLM        DELATTRE LEVIVIER MAROC        1 750 000        4000   
1     TIM                          TIMAR          301 100       38500   
2     AGM                           AGMA          200 000    6 000,00   
3     BAL                         BALIMA        1 744 000       15500   
4     CDM                            CDM       10 881 214       74700   
5     REB                  REBAB COMPANY          176 456        9350   
6     UMR                         UNIMER       11 413 880       15990   
7     ZDJ                   ZELLIDJA S.A          572 849        8800   
8     ADH              DOUJA PROM ADDOHA      402 551 254         990   
9     ADI                      ALLIANCES       22 078 588        8080   
10    ATL                   ATLANTASANAD       60 283 595       12400   
11    ATW              ATTIJARIWAFA BANK      215 140 839       44300   
12    BCP                            BCP      203 3

In [81]:
# Set up the Selenium driver
driver = webdriver.Chrome()

# Open a website
driver.get("https://www.casablanca-bourse.com/fr/live-market/instruments/AFM")

# Find and click the "Informations sur l'émetteur" link/button
time.sleep(1)
button = driver.find_element(By.LINK_TEXT, "Informations sur l'émetteur")
driver.execute_script("arguments[0].click();", button)
time.sleep(2)
page_source = driver.page_source

# Close the browser
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find the table or relevant elements in the parsed HTML
tables = soup.find_all('table')

# Extract firt
df_actionnaires_table = pd.read_html(str(tables[0]), flavor='bs4')[0]

# Set the columns names
df_actionnaires_table.columns = ["Principaux actionnaires", "Percentage"]
print(df_actionnaires_table)

# Extract second table
df_Info_table = pd.read_html(str(tables[1]), flavor='bs4')[0]

# Set the columns names
df_Info_table.columns = ["Informations sur la société", "value"]
print(df_Info_table)

# Extract third table
df_Chiffres_table = pd.read_html(str(tables[2]), flavor='bs4')[0]

# Set the columns names
df_Chiffres_table.columns = ["Chiffres clés", "2022", "2021", "2020"]
print(df_Chiffres_table)

# Extract fourth table
df_Ratios_table = pd.read_html(str(tables[3]), flavor='bs4')[0]

# Set the columns names
df_Ratios_table.columns = ["Ratios", "2022", "2021", "2020"]
print(df_Ratios_table)

# Extract five table
df_Dividende_table = pd.read_html(str(tables[5]), flavor='bs4')[0]

# Set the columns names
df_Dividende_table.columns = ["Année", "Montant", "Dividende", "Type dividende", "Date de détachement"]
print(df_Dividende_table)

# Extract sixth table
df_Franchi_table = pd.read_html(str(tables[7]), flavor='bs4')[0]

# Set the columns names
df_Franchi_table.columns = ["Date", "Déclarant", "Quantité", "Cours (MAD)", "Seuil Franchi en %", "Sens", "Intention du Déclarant"]
print(df_Franchi_table)

   Principaux actionnaires Percentage
0           TENOR FINANCES    39,48 %
1      DIVERS ACTIONNAIRES    25,80 %
2                     CIMR    11,63 %
3            FIPAR HOLDING    10,00 %
4              TENOR GROUP     5,53 %
5      TENOR INTERNATIONAL     5,00 %
6          TENOR ASSURANCE     2,56 %
7            BENSAÏD Farid     0,00 %
8            CAPITAL UNION     0,00 %
9         TENOR MANAGEMENT     0,00 %
10                   Total   100,00 %
  Informations sur la société  \
0           Nom de la société   
1                Siège social   
2     Commissaire aux comptes   
3        Date de constitution   
4         Date d'introduction   
5  Durée de l'Exercice Social   
6                Objet social   

                                               value  
0                                               AFMA  
1                        22 Boulevard Moulay Youssef  
2                         KPMG / Afric Audit Conseil  
3                                         01/01/1953  
4   

In [57]:
# Set up the Selenium driver
driver = webdriver.Chrome()

# Open a website
driver.get("https://www.casablanca-bourse.com/fr/live-market/instruments/AFM")

# Find and click the "Informations sur l'émetteur" link/button
button = driver.find_element(By.LINK_TEXT, "Informations sur l'émetteur")
button.click()
time.sleep(2)
# Extract Principaux actionnaires
first_table = driver.find_element(By.XPATH, "//table[@class='w-full']")
first_table_html = first_table.get_attribute('outerHTML')
df_first_table = pd.read_html(first_table_html)[0]

# Set the columns names
df_first_table.columns = ["Principaux actionnaires", "Percentage"]
print(df_first_table)

# Extract Indicateurs clés
# 1 Informations sur la société
second_table = driver.find_element(By.XPATH, "//div[@class='rounded-lg print:rounded-none shadow-sm border bg-gray-800 print:bg-white border-gray-600 print:border-gray-200 w-full overflow-x-scroll overflow-hidden']//table[@class='w-full']")
second_table_html = second_table.get_attribute('outerHTML')
df_second_table = pd.read_html(second_table_html)[0]

# Set the columns names
df_second_table.columns = ["Informations sur la société", "value"]
print(df_second_table)
# Perform actions after clicking the button
# ...

# Close the browser
driver.quit()


   Principaux actionnaires Percentage
0           TENOR FINANCES    39,48 %
1      DIVERS ACTIONNAIRES    25,80 %
2                     CIMR    11,63 %
3            FIPAR HOLDING    10,00 %
4              TENOR GROUP     5,53 %
5      TENOR INTERNATIONAL     5,00 %
6          TENOR ASSURANCE     2,56 %
7            BENSAÏD Farid     0,00 %
8            CAPITAL UNION     0,00 %
9         TENOR MANAGEMENT     0,00 %
10                   Total   100,00 %
   Informations sur la société     value
0               TENOR FINANCES   39,48 %
1          DIVERS ACTIONNAIRES   25,80 %
2                         CIMR   11,63 %
3                FIPAR HOLDING   10,00 %
4                  TENOR GROUP    5,53 %
5          TENOR INTERNATIONAL    5,00 %
6              TENOR ASSURANCE    2,56 %
7                BENSAÏD Farid    0,00 %
8                CAPITAL UNION    0,00 %
9             TENOR MANAGEMENT    0,00 %
10                       Total  100,00 %
